In [1]:
%matplotlib inline

import matplotlib.pyplot as plt
import numpy as np
import pandas as pd
import seaborn as sns
from datetime import datetime as dt
from sklearn.metrics import *

### ELECCIÓN DE FEATURES 

1. Porcentaje de victorias Local y Visitante ✅
2. Puntos por partido Local y Visitante
3. Porcentaje acierto tiros de 2 (Para Local y Visitante)
4. Porcentaje acierto tiros de 3 (Para Local y Visitante)
5. Porcentaje acierto tiros libres (Para Local y Visitante)
6. Promedio de Rebotes Totales Local y Visitante
7. Promedio de Asistencias Local y Visitante
8. Días libres Local y Visitante
9. Distancia de viajes
10. Lesión local y Lesión visitante

### 1. PORCENTAJE DE VICTORIAS

In [2]:
GAME_COLS  =['GAME_DATE_EST', 'GAME_ID', 'HOME_TEAM_ID',
       'VISITOR_TEAM_ID', 'SEASON', 'HOME_TEAM_WINS']
df = pd.read_csv("input/games.csv",usecols = GAME_COLS,parse_dates=["GAME_DATE_EST"],infer_datetime_format=True)
df = df.drop_duplicates().sort_values("GAME_DATE_EST").set_index(["GAME_DATE_EST"])
print(df.shape)
df.head()

(23195, 5)


,GAME_ID,HOME_TEAM_ID,VISITOR_TEAM_ID,SEASON,HOME_TEAM_WINS
GAME_DATE_EST,,,,,
2003-10-05,10300001,1610612762,1610612742,2003,1
2003-10-06,10300002,1610612763,1610612749,2003,1
2003-10-07,10300009,1610612758,1610612746,2003,1
2003-10-07,10300005,1610612757,1610612745,2003,1
2003-10-07,10300007,1610612748,1610612755,2003,1


In [3]:
df_ranking = pd.read_csv("input/ranking.csv",parse_dates=["STANDINGSDATE"])
df_ranking.sort_values("STANDINGSDATE",inplace=True)
print(df_ranking.shape)
 
df_ranking.drop(["CONFERENCE","LEAGUE_ID","HOME_RECORD","ROAD_RECORD"],axis=1,inplace=True) 

df_ranking.head()

(179533, 12)


,TEAM_ID,SEASON_ID,STANDINGSDATE,TEAM,G,W,L,W_PCT
150747,1610612739,22002,2003-10-01,Cleveland,82,17,65,0.207
150746,1610612761,22002,2003-10-01,Toronto,82,24,58,0.293
150734,1610612751,22002,2003-10-01,New Jersey,82,49,33,0.598
150735,1610612754,22002,2003-10-01,Indiana,82,48,34,0.585
150736,1610612755,22002,2003-10-01,Philadelphia,82,48,34,0.585


In [4]:
print(df.shape)

df_ranking.set_index("STANDINGSDATE",inplace=True)

df = pd.merge_asof(df, df_ranking.add_suffix("_homeTeam"), left_index=True, right_index=True, left_by="HOME_TEAM_ID", right_by='TEAM_ID'+"_homeTeam", allow_exact_matches=False)

df = pd.merge_asof(df, df_ranking.add_suffix("_awayTeam"), left_index=True, right_index=True, left_by="VISITOR_TEAM_ID", right_by='TEAM_ID'+"_awayTeam", allow_exact_matches=False)

df.drop(["SEASON_ID_awayTeam","TEAM_ID_awayTeam","TEAM_ID_homeTeam"],axis=1,inplace=True) 
df.rename(columns={"SEASON_ID_homeTeam":"SEASON_ID"},inplace=True)
print(df.shape)
df.sort_values(by='SEASON', ascending=False)

(23195, 5)
(23195, 16)


,GAME_ID,HOME_TEAM_ID,VISITOR_TEAM_ID,SEASON,HOME_TEAM_WINS,SEASON_ID,TEAM_homeTeam,G_homeTeam,W_homeTeam,L_homeTeam,W_PCT_homeTeam,TEAM_awayTeam,G_awayTeam,W_awayTeam,L_awayTeam,W_PCT_awayTeam
GAME_DATE_EST,,,,,,,,,,,,,,,,
2020-03-01,21900895,1610612766,1610612749,2019,0,22019,Charlotte,59,21,38,0.356,Milwaukee,59,51,8,0.864
2019-12-02,21900294,1610612755,1610612762,2019,1,22019,Philadelphia,20,14,6,0.700,Utah,20,12,8,0.600
2019-11-27,21900259,1610612745,1610612748,2019,1,22019,Houston,17,11,6,0.647,Miami,16,12,4,0.750
2019-11-27,21900264,1610612740,1610612747,2019,0,22019,New Orleans,17,6,11,0.353,L.A. Lakers,17,15,2,0.882
2019-11-27,21900263,1610612756,1610612764,2019,0,22019,Phoenix,16,8,8,0.500,Washington,15,5,10,0.333
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
2004-02-23,20300812,1610612739,1610612740,2003,1,22003,Cleveland,56,22,34,0.393,New Orleans,55,30,25,0.545
2004-02-24,20300821,1610612759,1610612745,2003,1,22003,San Antonio,56,37,19,0.661,Houston,55,32,23,0.582
2004-02-24,20300816,1610612754,1610612744,2003,1,22003,Indiana,56,41,15,0.732,Golden State,54,24,30,0.444


In [5]:
df = df[df.G_homeTeam!=0]

In [6]:
df.head(20)

,GAME_ID,HOME_TEAM_ID,VISITOR_TEAM_ID,SEASON,HOME_TEAM_WINS,SEASON_ID,TEAM_homeTeam,G_homeTeam,W_homeTeam,L_homeTeam,W_PCT_homeTeam,TEAM_awayTeam,G_awayTeam,W_awayTeam,L_awayTeam,W_PCT_awayTeam
GAME_DATE_EST,,,,,,,,,,,,,,,,
2003-10-05,10300001,1610612762,1610612742,2003,1,22002,Utah,82,47,35,0.573,Dallas,82,60,22,0.732
2003-10-30,20300018,1610612756,1610612739,2003,1,22003,Phoenix,1,0,1,0.000,Cleveland,1,0,1,0.000
2003-10-30,20300016,1610612753,1610612740,2003,0,22003,Orlando,1,1,0,1.000,New Orleans,1,1,0,1.000
2003-10-31,20300024,1610612741,1610612737,2003,1,22003,Chicago,1,0,1,0.000,Atlanta,1,0,1,0.000
2003-10-31,20300021,1610612754,1610612749,2003,0,22003,Indiana,1,1,0,1.000,Milwaukee,1,0,1,0.000
2003-10-31,20300025,1610612746,1610612760,2003,0,22003,L.A. Clippers,1,0,1,0.000,Seattle,1,1,0,1.000
2003-10-31,20300022,1610612751,1610612750,2003,1,22003,New Jersey,1,0,1,0.000,Minnesota,1,1,0,1.000
2003-10-31,20300019,1610612761,1610612764,2003,1,22003,Toronto,1,1,0,1.000,Washington,1,1,0,1.000
2003-10-31,20300026,1610612758,1610612755,2003,1,22003,Sacramento,1,1,0,1.000,Philadelphia,1,1,0,1.000


In [7]:
df['VICTORIA']= df['HOME_TEAM_WINS'].replace({0: 'VISITANTE', 1: 'LOCAL'})

In [8]:
df.drop(columns=['HOME_TEAM_WINS'])

,GAME_ID,HOME_TEAM_ID,VISITOR_TEAM_ID,SEASON,SEASON_ID,TEAM_homeTeam,G_homeTeam,W_homeTeam,L_homeTeam,W_PCT_homeTeam,TEAM_awayTeam,G_awayTeam,W_awayTeam,L_awayTeam,W_PCT_awayTeam,VICTORIA
GAME_DATE_EST,,,,,,,,,,,,,,,,
2003-10-05,10300001,1610612762,1610612742,2003,22002,Utah,82,47,35,0.573,Dallas,82,60,22,0.732,LOCAL
2003-10-30,20300018,1610612756,1610612739,2003,22003,Phoenix,1,0,1,0.000,Cleveland,1,0,1,0.000,LOCAL
2003-10-30,20300016,1610612753,1610612740,2003,22003,Orlando,1,1,0,1.000,New Orleans,1,1,0,1.000,VISITANTE
2003-10-31,20300024,1610612741,1610612737,2003,22003,Chicago,1,0,1,0.000,Atlanta,1,0,1,0.000,LOCAL
2003-10-31,20300021,1610612754,1610612749,2003,22003,Indiana,1,1,0,1.000,Milwaukee,1,0,1,0.000,VISITANTE
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
2020-03-01,21900900,1610612740,1610612747,2019,22019,New Orleans,59,26,33,0.441,L.A. Lakers,58,45,13,0.776,VISITANTE
2020-03-01,21900899,1610612758,1610612765,2019,22019,Sacramento,59,25,34,0.424,Detroit,61,20,41,0.328,LOCAL
2020-03-01,21900898,1610612743,1610612761,2019,22019,Denver,59,40,19,0.678,Toronto,59,42,17,0.712,LOCAL


In [9]:
#df.to_csv('output/percentageWin.csv')

In [10]:
df.shape

(22624, 17)